<a href="https://colab.research.google.com/github/aleks-haksly/leetcode/blob/main/pandas/pandas_easy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

1179. Reformat Department Table

https://leetcode.com/problems/reformat-department-table/description/

In [ ]:
import pandas as pd

def reformat_table(department: pd.DataFrame) -> pd.DataFrame:
    all_mths = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
    pvt = department.pivot_table(index='id', columns='month', values='revenue').reindex(columns = all_mths)
    pvt.columns = [_+"_Revenue" for _ in pvt.columns]
    return pvt.reset_index()

175. Combine Two Tables

https://leetcode.com/problems/combine-two-tables/description/

In [3]:
data = [[1, 'Wang', 'Allen'], [2, 'Alice', 'Bob']]
person = pd.DataFrame(data, columns=['personId', 'firstName', 'lastName']).astype({'personId':'Int64', 'firstName':'object', 'lastName':'object'})
data = [[1, 2, 'New York City', 'New York'], [2, 3, 'Leetcode', 'California']]
address = pd.DataFrame(data, columns=['addressId', 'personId', 'city', 'state']).astype({'addressId':'Int64', 'personId':'Int64', 'city':'object', 'state':'object'})

```sql
SELECT
  p.firstName,
  p.lastName,
  a.city,
  a.state
FROM person p
LEFT JOIN Address a
ON p.personId = a.personId

```

In [6]:
person.merge(address, how='left', on='personId')[['firstName', 'lastName', 'city','state' ]]

,firstName,lastName,city,state
0,Wang,Allen,NaN,NaN
1,Alice,Bob,New York City,New York


181. Employees Earning More Than Their Managers

https://leetcode.com/problems/employees-earning-more-than-their-managers/description/

In [25]:
data = [[1, 'Joe', 70000, 3], [2, 'Henry', 80000, 4], [3, 'Sam', 60000, None], [4, 'Max', 90000, None]]
employee = pd.DataFrame(data, columns=['id', 'name', 'salary', 'managerId']).astype({'id':'Int64', 'name':'object', 'salary':'Int64', 'managerId':'Int64'})

```sql
SELECT e2.name as  Employee FROM Employee e1
INNER JOIN Employee e2 on e1.id = e2.managerId
WHERE e1.salary < e2.salary
```


In [26]:
def find_employees(employee: pd.DataFrame) -> pd.DataFrame:
    merged = employee.merge(employee, how='inner', left_on='id', right_on='managerId')
    merged = merged[merged.salary_x < merged.salary_y][['name_y']].rename(columns={'name_y':'Employee'})
    return merged

182. Duplicate Emails

https://leetcode.com/problems/duplicate-emails/description/

```sql
SELECT email as "Email" FROM Person
GROUP BY email
HAVING count(email) > 1
```

In [28]:
data = [[1, 'a@b.com'], [2, 'c@d.com'], [3, 'a@b.com']]
person = pd.DataFrame(data, columns=['id', 'email']).astype({'id':'Int64', 'email':'object'})

In [51]:
def duplicate_emails(person: pd.DataFrame) -> pd.DataFrame:
    return person.groupby('email', as_index=False).count().query("id > 1", engine="python")[['email']]

183. Customers Who Never Order

https://leetcode.com/problems/customers-who-never-order/description/

```sql
SELECT name as "Customers" FROM Customers c WHERE c.id NOT IN (SELECT customerId FROM Orders )
```

In [52]:
data = [[1, 'Joe'], [2, 'Henry'], [3, 'Sam'], [4, 'Max']]
customers = pd.DataFrame(data, columns=['id', 'name']).astype({'id':'Int64', 'name':'object'})
data = [[1, 3], [2, 1]]
orders = pd.DataFrame(data, columns=['id', 'customerId']).astype({'id':'Int64', 'customerId':'Int64'})

In [64]:
def find_customers(customers: pd.DataFrame, orders: pd.DataFrame) -> pd.DataFrame:
    return customers[~customers['id']\
                     .isin(orders.customerId)][['name']]\
                     .rename({'name': 'Customers'}, axis=1)

196. Delete Duplicate Emails

https://leetcode.com/problems/delete-duplicate-emails/description/

```sql
DELETE FROM Person WHERE id in (
SELECT p1.id FROM Person p1
JOIN Person p2 ON p1.email = p2.email and p1.id > p2.id)
```


In [67]:
data = [[1, 'john@example.com'], [2, 'bob@example.com'], [3, 'john@example.com']]
person = pd.DataFrame(data, columns=['id', 'email']).astype({'id':'int64', 'email':'object'})

In [81]:
def delete_duplicate_emails(person: pd.DataFrame) -> None:
    person.sort_values(by='id',ascending=True,inplace=True)
    person.drop_duplicates(subset='email', keep='first', inplace=True)

197. Rising Temperature

https://leetcode.com/problems/rising-temperature/description/

```sql

SELECT id as Id FROM (
SELECT *,
lag(temperature, 1) OVER (ORDER BY recordDate) as tlag,
lag(recordDate , 1) OVER (ORDER BY recordDate) as dlag
FROM Weather) t
WHERE temperature - tlag > 0 and recordDate - dlag = 1
```

In [94]:
data = [[1, '2015-01-01', 10], [2, '2015-01-02', 25], [3, '2015-01-03', 20], [4, '2015-01-04', 30]]
weather = pd.DataFrame(data, columns=['id', 'recordDate', 'temperature']).astype({'id':'Int64', 'recordDate':'datetime64[ns]', 'temperature':'Int64'})

In [117]:
def rising_temperature(weather: pd.DataFrame) -> pd.DataFrame:
    weather = weather.sort_values(by='recordDate')
    df = weather[(weather['temperature'].diff()>0) & (weather['recordDate'].diff().dt.days == 1)]
    return df[['id']]

511. Game Play Analysis I

https://leetcode.com/problems/game-play-analysis-i/description/

```sql
SELECT player_id,
min(event_date) as first_login
FROM Activity
GROUP BY player_id
```

In [125]:
data = [[1, 2, '2016-03-01', 5], [1, 2, '2016-05-02', 6], [2, 3, '2017-06-25', 1], [3, 1, '2016-03-02', 0], [3, 4, '2018-07-03', 5]]
activity = pd.DataFrame(data, columns=['player_id', 'device_id', 'event_date', 'games_played']).astype({'player_id':'Int64', 'device_id':'Int64', 'event_date':'datetime64[ns]', 'games_played':'Int64'})

In [126]:
activity

,player_id,device_id,event_date,games_played
0,1,2,2016-03-01,5
1,1,2,2016-05-02,6
2,2,3,2017-06-25,1
3,3,1,2016-03-02,0
4,3,4,2018-07-03,5


In [133]:
activity.groupby('player_id')['event_date'].min().reset_index().rename({'event_date':'first_login'}, axis =1)

,player_id,first_login
0,1,2016-03-01
1,2,2017-06-25
2,3,2016-03-02


577. Employee Bonus

https://leetcode.com/problems/employee-bonus/description/

```sql
SELECT
e.name,
b.bonus
FROM Employee e
LEFT JOIN Bonus b on b.empId = e.empId
WHERE COALESCE(b.bonus , 0) < 1000
```

```
DataFrame.eq
Compare DataFrames for equality elementwise.

DataFrame.ne
Compare DataFrames for inequality elementwise.

DataFrame.le
Compare DataFrames for less than inequality or equality elementwise.

DataFrame.lt
Compare DataFrames for strictly less than inequality elementwise.

DataFrame.ge
Compare DataFrames for greater than inequality or equality elementwise.

DataFrame.gt
Compare DataFrames for strictly greater than inequality elementwise.

```

In [134]:
data = [[3, 'Brad', None, 4000], [1, 'John', 3, 1000], [2, 'Dan', 3, 2000], [4, 'Thomas', 3, 4000]]
employee = pd.DataFrame(data, columns=['empId', 'name', 'supervisor', 'salary']).astype({'empId':'Int64', 'name':'object', 'supervisor':'Int64', 'salary':'Int64'})
data = [[2, 500], [4, 2000]]
bonus = pd.DataFrame(data, columns=['empId', 'bonus']).astype({'empId':'Int64', 'bonus':'Int64'})

In [143]:
def employee_bonus(employee: pd.DataFrame, bonus: pd.DataFrame) -> pd.DataFrame:
    merged = employee.merge(bonus, how='left', on='empId')
    return merged[merged['bonus'].lt(1000) | merged['bonus'].isnull()][['name', 'bonus']]

,name,bonus
0,Brad,<NA>
1,John,<NA>
2,Dan,500
